In [11]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm  # 进度条工具（可选）
from pathlib import Path
from datetime import datetime, timedelta
import re

# 1. 处理lastprice数据

In [12]:
# 1. 生成交易时间列表（间隔1秒）
def generate_trading_hours():
    morning_start = datetime.strptime("09:00:00", "%H:%M:%S").time()
    morning_end = datetime.strptime("11:30:00", "%H:%M:%S").time()
    afternoon_start = datetime.strptime("13:30:00", "%H:%M:%S").time()
    afternoon_end = datetime.strptime("14:59:59", "%H:%M:%S").time()
    
    time_list = []
    current_time = morning_start
    while current_time <= morning_end:
        time_list.append(current_time.strftime("%H:%M:%S"))
        current_time = (datetime.combine(datetime.min, current_time) + 
                       timedelta(seconds=1)).time()
    
    current_time = afternoon_start
    while current_time <= afternoon_end:
        time_list.append(current_time.strftime("%H:%M:%S"))
        current_time = (datetime.combine(datetime.min, current_time) + 
                       timedelta(seconds=1)).time()
    
    return time_list

def process_tick_file(file_path):
    
    df = pd.read_csv(file_path)

    # 1. 用ffill填充缺失值
    df = df.ffill()

    # 2. 创建lastPrice列（LastPrice向下平移一行）
    df['lastPrice'] = df['LastPrice'].shift(1)

    # 3. 保留每个UpdateTime对应UpdateMillisec最大的行
    df = df.loc[df.groupby('UpdateTime')['UpdateMillisec'].idxmin()]

    # 5. 筛选交易时间内的数据
    df['UpdateTime'] = pd.to_datetime(df['UpdateTime'], format='%H:%M:%S').dt.time

    df = df[df['UpdateTime'].apply(lambda x: x.strftime("%H:%M:%S") in trading_hours)]
    df_result = pd.DataFrame({'ts': trading_hours})

    df_result = pd.merge(df_result, df[['UpdateTime', 'InstrumentID', 'lastPrice','BidPrice1','AskPrice1']], how='left', left_on='ts', right_on=df['UpdateTime'].apply(lambda x: x.strftime("%H:%M:%S")))
    df_result = df_result.reset_index(drop=True)
    df_result = df_result.ffill()
    df_result = df_result[['ts', 'InstrumentID','lastPrice','BidPrice1','AskPrice1']]
    df_result['trading_date'] = re.search(r'(\d{8})', os.path.basename(file_path)).group(1)
    df_result.columns = ['ts', 'InstrumentID', 'lastPrice','BidPrice1','AskPrice1','trading_date']
    df_result = df_result[['trading_date', 'ts', 'InstrumentID','lastPrice','BidPrice1','AskPrice1']]

    # 输出处理后的DataFrame
    #print(df_result)
    return df_result


# 2. 计算固定参数的指标

## 1.布林带相关（upperBound, lowerBound, midBound） 2.EMA （shortEMA，longEMA）

In [13]:
def calculate_technical_indicators(df, params):
    """
    计算技术指标并添加到原始DataFrame
    参数：
        df: 包含ts和lastPrice的DataFrame
        params: 参数字典，包含各种技术指标参数
    返回：
        添加了技术指标列的DataFrame（按InstrumentID分组计算后合并）
    """
    shortEmaLookback = params['shortEmaLookback'][0]
    longEmaLookback = params['longEmaLookback'][0]
    stopLossBase = params['stopLossBase'][0]
    stopLossMult = params['stopLossMult'][0]
    ema_short = shortEmaLookback*60
    ema_long = longEmaLookback*60
    bollinger_std_multiplier=2
    
    # 定义计算技术指标的函数（将在每个分组上应用）
    def calculate_indicators(group):
        # 1. 计算6000秒均线（使用滚动窗口）
        group['mid_band'] = group['lastPrice'].rolling(window=6000, min_periods=1).mean()

        # 2. 计算布林带（中轨为6000秒均线）
        rolling_std = group['lastPrice'].rolling(window=6000, min_periods=1).std(ddof=1)
        group['upper_band'] = group['mid_band'] + (rolling_std * bollinger_std_multiplier)
        group['lower_band'] = group['mid_band'] - (rolling_std * bollinger_std_multiplier)
        
        # 3. 计算EMA（指数移动平均）
        group['shortEMA'] = group['lastPrice'].ewm(span=ema_short, adjust=False).mean()
        group['longEMA'] = group['lastPrice'].ewm(span=ema_long, adjust=False).mean()
        
        return group
    
    # 按InstrumentID分组并应用计算函数
    grouped_df = df.groupby('InstrumentID', group_keys=False).apply(calculate_indicators)
    
    return grouped_df

# 2. 计算小趋势分

In [14]:
import pandas as pd
import numpy as np

def calculate_short_trend_score(df, params,index) -> pd.DataFrame:
    """
    根据5分钟K线计算秒级的shortTrendScore。
    """
    openMinBase = params['openMinBase'][0]
    openMinMult = params['openMinMult'][index]
    shortTrendAlloc = params['shortTrendAlloc'][0]

    # 2. 创建DatetimeIndex
    '''
    print("正在构建真实时间索引...")
    unique_dates = pd.Series(df['trading_date'].unique()).sort_values().reset_index(drop=True)
    prev_date_map = pd.Series(unique_dates.shift(1).values, index=unique_dates.values)
    calendar_date_series = df['trading_date'].map(prev_date_map)
    calendar_date_int = np.where(df['session_part'] == 0, calendar_date_series, df['trading_date'])
    s_calendar_date = pd.Series(calendar_date_int)
    df['datetime'] = pd.to_datetime(s_calendar_date.astype(str) + ' ' + df['ts'], errors='coerce', format='%Y%m%d.0 %H:%M:%S')
    '''
    df['datetime'] = pd.to_datetime(df['trading_date'].astype(str) + ' ' + df['ts'],
    format='%Y%m%d %H:%M:%S',  # 移除了".0"部分
    errors='coerce'
)
   
    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)
    #print(df.index.isnull().sum())

    # 3. 创建5分钟OHLC K线
    print("正在创建5分钟K线...")
    ohlc_5min = df['lastPrice'].resample('5min').ohlc()
    ohlc_5min.dropna(inplace=True)

    # 4. 准备用于向量化计算的数据
    print("正在准备数据以便计算...")
    df['kline_ts'] = df.index.floor('5min')

    kline_data = ohlc_5min[['open', 'close']].copy()
    kline_data.rename(columns={'open': 'K1O'}, inplace=True)
    for i in range(2, 6):
        kline_data[f'K{i}O'] = ohlc_5min['open'].shift(i - 1)
        kline_data[f'K{i}C'] = ohlc_5min['close'].shift(i - 1)

    cols_to_merge = ['K1O'] + [f'K{i}O' for i in range(2, 6)] + [f'K{i}C' for i in range(2, 4)]
    df = pd.merge(df, kline_data[cols_to_merge], left_on='kline_ts', right_index=True, how='left')

    # 5. 计算 S_Trend
    print("正在计算S_Trend...")
    
    # 关键点：K1L 是实时秒级价格，其他K线值是5分钟级别的。
    K1L = df['lastPrice']
    K1O = df['K1O']
    K2O, K2C = df['K2O'], df['K2C']
    K3O, K3C = df['K3O'], df['K3C']
    K4O = df['K4O']
    K5O = df['K5O']
    
    # openMin 的计算本身就依赖于实时的 LastPrice (K1L)
    openMin = np.maximum(openMinBase * K1L, df['ATR'] * openMinMult)

    # --- 上涨趋势逻辑 (S_Trend = 1) ---
    threshold_long_1 = 2 * openMin * 1
    threshold_long_2 = openMin * 1
    
    long_case1 = ((K1L - K1O) >= 0) & ((K1L - K2O) >= threshold_long_1)
    long_case2_conds = (((K1L - K1O) >= 0) & ((K2C - K2O) >= 0) & ((K3C - K3O) >= 0))
    long_case2_triggers = (((K1L - K3O) >= threshold_long_2) | ((K1L - K4O) >= threshold_long_2) | ((K1L - K5O) >= threshold_long_2))
    is_long_trend = long_case1 | (long_case2_conds & long_case2_triggers)

    # --- 下跌趋势逻辑 (S_Trend = -1) ---
    threshold_short_1 = 2 * openMin * 1
    threshold_short_2 = openMin * 1
    
    short_case1 = ((K1O - K1L) >= 0) & ((K2O - K1L) >= threshold_short_1)
    short_case2_conds = (((K1O - K1L) >= 0) & ((K2O - K2C) >= 0) & ((K3O - K3C) >= 0))
    short_case2_triggers = (((K3O - K1L) >= threshold_short_2) | ((K4O - K1L) >= threshold_short_2) | ((K5O - K1L) >= threshold_short_2))
    is_short_trend = short_case1 | (short_case2_conds & short_case2_triggers)

    # 组合逻辑，创建S_Trend列
    df['S_Trend'] = np.select(
        [is_long_trend, is_short_trend],
        [1, -1],
        default=0
    )

    # 6. 计算最终的 shortTrendScore
    print("正在计算shortTrendScore...")
    df['shortTrendScore'] = df['S_Trend'] * shortTrendAlloc
    df.reset_index(inplace=True,drop=True)
    
    print("计算完成。")
    return df

# 2. 计算costscore

In [15]:
def calculate_cost_score(df, params, index):
    """
    计算成本评分，修复了np.max使用问题
    参数:
        df: 包含必要数据的DataFrame
        params: 参数字典
    返回:
        添加了cost_score列的DataFrame
    """
    costAlloc = params['costAlloc'][index]
    bandLimitBase = params['bandLimitBase'][0]
    bandLimitMult = params['bandLimitMult'][index]
    
    # 计算band_limit - 使用np.maximum替代np.max
    df['band_limit'] = np.maximum(
        bandLimitBase * df['lastPrice'], 
        bandLimitMult * df['ATR']
    )
    
    # 筛选有效行
    valid_mask = (~df['lower_band'].isna()) & (~df['mid_band'].isna()) & (~df['upper_band'].isna())
    valid_rows = df[valid_mask].copy()
    
    # 计算位置评分
    band_width = valid_rows['upper_band'] - valid_rows['lower_band']
    mid = valid_rows['mid_band']
    close = valid_rows['lastPrice']
    
    pos_v = np.select(
        [
            close <= valid_rows['lower_band'],
            (valid_rows['lower_band'] < close) & (close <= mid),
            (mid < close) & (close <= valid_rows['upper_band']),
            close > valid_rows['upper_band']
        ],
        [
            costAlloc,
            costAlloc - costAlloc * (mid - close) / (band_width / 2),
            -costAlloc * (close - mid) / (band_width / 2),
            -costAlloc
        ],
        default=0.0
    )
    
    # 计算宽度乘数
    mult = np.minimum(band_width / valid_rows['band_limit'], 2)
    mult[valid_rows['band_limit'] <= 0] = 0.0
    
    # 计算最终cost_score
    df['cost_score'] = 0.0  # 初始化列
    df.loc[valid_mask, 'cost_score'] = np.where(
        pos_v > 0,
        np.minimum(pos_v * mult, costAlloc),
        np.maximum(pos_v * mult, -costAlloc)
    )
    
    return df

# 4. 计算midscore

In [16]:
def calculate_mid_trend(df, params, index) :
    """
    计算中趋势方向和得分并添加到DataFrame
    
    参数:
        df: 包含必要数据的DataFrame
        params: 参数字典，包含trendMultMid和midTrendAlloc等参数
        
    返回:
        添加了'M_trend'和'M_Trend_v'列的DataFrame
    """
    # 从params中提取参数
    trendMultMid = params['trendMultMid'][0]
    midTrendAlloc = params['midTrendAlloc'][0]
    trendDiffBase = params['trendDiffBase'][index]
    trendDiffMult = params['trendDiffMult'][0]
    
    # 计算差异
    diff = df['shortEMA'] - df['longEMA']
    df['trendDiff'] = np.maximum(trendDiffBase * df['lastPrice'], df['ATR'] * trendDiffMult)
    #print(df['trendDiff'])
    threshold = df['trendDiff']
    
    # 使用向量化操作计算M_trend
    conditions = [
        diff > trendMultMid * threshold,
        (threshold <= diff) & (diff <= trendMultMid * threshold),
        diff < -1 * trendMultMid * threshold,
        (-1 * trendMultMid * threshold <= diff) & (diff <= -1 * threshold)
    ]
    choices = [2, 1, -2, -1]
    df['M_trend'] = np.select(conditions, choices, default=0)
    
    # 使用向量化操作计算M_Trend_v
    trend_conditions = [
        df['M_trend'] == 2,
        df['M_trend'] == 1,
        df['M_trend'] == -1,
        df['M_trend'] == -2
    ]
    trend_choices = [
        midTrendAlloc,
        0.5 * midTrendAlloc,
        -0.5 * midTrendAlloc,
        -1 * midTrendAlloc
    ]
    df['midTrendScore'] = np.select(trend_conditions, trend_choices, default=0)
    
    return df

# 5. 计算总得分

In [17]:
def add_position_and_cost(df, params, index):
    """
    为DataFrame添加pos和cost列，跟踪持仓状态和开仓成本
    
    参数:
        df: 包含交易数据的DataFrame
        params: 参数字典，包含openThreshold等参数
        
    返回:
        添加了pos和cost列的DataFrame
    """
    # 初始化新列
    df['pos'] = 0  # 0表示空仓，1表示持多仓，-1表示持空仓
    df['cost'] = np.nan  # 开仓成本价
    df['best_price'] = np.nan  # 用于动态止损的最高/最低价
    
    # 从params中提取参数
    openThreshold = params['openThreshold'][0]
    closeDay = '14:59:00'
    closeNight = '22:59:00'
    stopLossBase = params['stopLossBase'][0]
    stopLossMult = params['stopLossMult'][0]
    fullAlloc = params['shortTrendAlloc'][0] + params['costAlloc'][index] + params['midTrendAlloc'][0]
    
    # 初始化持仓状态变量
    current_position = 0
    current_cost = np.nan
    current_best_price = np.nan
    
    for i in range(len(df)):
        row = df.iloc[i]
        
        # 检查是否在平仓时间
        current_time = row['ts']
        is_close_time = (current_time >= closeDay) or (current_time >= closeNight)
        
        # 平仓逻辑
        if current_position != 0 and is_close_time:
            df.at[i, 'pos'] = 0
            df.at[i, 'cost'] = np.nan
            df.at[i, 'best_price'] = np.nan
            current_position = 0
            current_cost = np.nan
            current_best_price = np.nan
            continue
        
        # 动态止损平仓逻辑
        if current_position == 1:  # 多头持仓
            current_best_price = max(current_best_price, row['lastPrice']) if not np.isnan(current_best_price) else row['lastPrice']
            stop_loss = max(stopLossBase * row['lastPrice'], row['ATR'] * stopLossMult)
            if current_best_price - row['lastPrice'] > stop_loss:
                df.at[i, 'pos'] = 0
                df.at[i, 'cost'] = np.nan
                df.at[i, 'best_price'] = np.nan
                current_position = 0
                current_cost = np.nan
                current_best_price = np.nan
                continue
                
        elif current_position == -1:  # 空头持仓
            current_best_price = min(current_best_price, row['lastPrice']) if not np.isnan(current_best_price) else row['lastPrice']
            stop_loss = max(stopLossBase * row['lastPrice'], row['ATR'] * stopLossMult)
            if row['lastPrice'] - current_best_price > stop_loss:
                df.at[i, 'pos'] = 0
                df.at[i, 'cost'] = np.nan
                df.at[i, 'best_price'] = np.nan
                current_position = 0
                current_cost = np.nan
                current_best_price = np.nan
                continue
        
        # 开仓逻辑
        if current_position == 0 and not np.isnan(row['totalScore']) and not is_close_time:
            if row['totalScore'] > openThreshold * fullAlloc:
                current_position = 1
                current_cost = row['AskPrice1']
                current_best_price = row['lastPrice']
            elif row['totalScore'] < -1 * openThreshold * fullAlloc:
                current_position = -1
                current_cost = row['BidPrice1']
                current_best_price = row['lastPrice']
        
        # 更新当前行的持仓状态和成本价
        df.at[i, 'pos'] = current_position
        df.at[i, 'cost'] = current_cost
        df.at[i, 'best_price'] = current_best_price if current_position != 0 else np.nan
    
    return df

In [ ]:
if __name__ == '__main__':
    # 定义参数
    params = {'kbar': [5],
                'shortEmaLookback': [30],   #ema
                'longEmaLookback': [180],   #ema

                'trendDiffMult': [0],
                'trendMultShort' : [2],
                'trendMultMid' : [2],
                'shortTrendAlloc' : [10],
                'midTrendAlloc' : [5],
                'openWait' : [1200],
                'bandLimitBase' : [0.001],
                'openMinBase' : [0.001],
                'unitSize' : [1],
                'openThreshold' : [0.5],
                'closeThreshold' : [-1],
                'stopLossBase' : [0.001],
                'stopLossMult' : [3],

                'trendDiffBase': [0.002,0.004,0.006],
                'bandLimitMult' : [2,2.5,3],
                'openMinMult' : [1,1.2,1.5],
                'costAlloc' : [15,10,5]
            }
    
    trading_hours = generate_trading_hours()
    print(trading_hours)

    # 手动指定待处理的品种列表（示例）
    pending_comds = ['lc','SA'] #  

    # 基础路径配置
    base_input_dir = "C:/Users/CYWY2K/HKUST/STUDY/summer/MAFS6100T/week5/data/2023-2025/"

    for comd in pending_comds:

        print(f"=================  Processing {comd} =================")
        # 1. 处理初始数据
        input_folder = os.path.join(base_input_dir, comd) 
        df_all = pd.DataFrame()  
        # 处理该品种的所有CSV文件
        for file in os.listdir(input_folder):
            if file.endswith(".csv"):
                file_path = os.path.join(input_folder, file)
                processed_df = process_tick_file(file_path)  # 假设已定义处理函数
                df_all = pd.concat([df_all, processed_df], ignore_index=True)
                #print(processed_df)

        print(f"已完成品种 {comd} 的数据处理")

        # 2. 计算基础技术指标
        df_with_indicators = calculate_technical_indicators(df_all, params)
        print(f"已完成品种 {comd} 的基础技术指标计算")

        # 3. 导入atr数据
        atr = pd.read_parquet(f'C:/Users/CYWY2K/HKUST/STUDY/summer/MAFS6100T/week5/processed_data/2023-2025/{comd}/{comd}_atr_1s_.parquet')
        print(f"已导入品种 {comd} 的ATR数据")

        # 4. 合并数据
        df = pd.merge(df_with_indicators, atr, on=['trading_date', 'ts'], how='left')
        df = df[['trading_date', 'ts', 'InstrumentID_x', 'lastPrice', 'mid_band', 
                'upper_band', 'lower_band', 'shortEMA', 'longEMA', 'ATR','BidPrice1','AskPrice1']]
        df.columns = ['trading_date', 'ts', 'InstrumentID', 'lastPrice', 'mid_band', 
                    'upper_band', 'lower_band', 'shortEMA', 'longEMA', 'ATR','BidPrice1','AskPrice1']
        print("合并后数据为：\n")
        print(df)
        print(f"已完成品种 {comd} 的数据合并")

        # 5. 计算不同参数组合下的得分和持仓
        for i in range(3):

            print(f"Processing parameter set {i}/3: {params['trendDiffBase'][i]}, {params['bandLimitMult'][i]}, {params['openMinMult'][i]}, {params['costAlloc'][i]}")

            file_path = f'C:/Users/CYWY2K/HKUST/STUDY/summer/MAFS6100T/week5/processed_data/2023-2025/{comd}/{comd}_all_ind_{i+1}.parquet'
            if os.path.exists(file_path):
                print(f"File {file_path} already exists. Skipping computation.")
                continue  # 如果文件已存在，跳过计算

            # 计算得分    
            df1 = calculate_short_trend_score(df, params, i)  
            df2 = calculate_cost_score(df1, params, i)
            df3 = calculate_mid_trend(df2, params, i)
            df3['totalScore'] = df3['shortTrendScore'] + df3['cost_score'] + df3['midTrendScore']
            df3 = df3[['trading_date','ts','InstrumentID','lastPrice','ATR', 'totalScore','BidPrice1','AskPrice1',]]
    
            # 计算持仓和成本
            df4 = add_position_and_cost(df3, params, i)
            df4.to_parquet(file_path)



['09:00:00', '09:00:01', '09:00:02', '09:00:03', '09:00:04', '09:00:05', '09:00:06', '09:00:07', '09:00:08', '09:00:09', '09:00:10', '09:00:11', '09:00:12', '09:00:13', '09:00:14', '09:00:15', '09:00:16', '09:00:17', '09:00:18', '09:00:19', '09:00:20', '09:00:21', '09:00:22', '09:00:23', '09:00:24', '09:00:25', '09:00:26', '09:00:27', '09:00:28', '09:00:29', '09:00:30', '09:00:31', '09:00:32', '09:00:33', '09:00:34', '09:00:35', '09:00:36', '09:00:37', '09:00:38', '09:00:39', '09:00:40', '09:00:41', '09:00:42', '09:00:43', '09:00:44', '09:00:45', '09:00:46', '09:00:47', '09:00:48', '09:00:49', '09:00:50', '09:00:51', '09:00:52', '09:00:53', '09:00:54', '09:00:55', '09:00:56', '09:00:57', '09:00:58', '09:00:59', '09:01:00', '09:01:01', '09:01:02', '09:01:03', '09:01:04', '09:01:05', '09:01:06', '09:01:07', '09:01:08', '09:01:09', '09:01:10', '09:01:11', '09:01:12', '09:01:13', '09:01:14', '09:01:15', '09:01:16', '09:01:17', '09:01:18', '09:01:19', '09:01:20', '09:01:21', '09:01:22', '09

C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\120890607.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = df.groupby('InstrumentID', group_keys=False).apply(calculate_indicators)


已完成品种 SA 的基础技术指标计算
已导入品种 SA 的ATR数据
合并后数据为：

        trading_date        ts InstrumentID  lastPrice     mid_band  \
0           20230103  09:00:00        SA305     2740.0  2740.000000   
1           20230103  09:00:01        SA305     2744.0  2742.000000   
2           20230103  09:00:02        SA305     2742.0  2742.000000   
3           20230103  09:00:03        SA305     2741.0  2741.750000   
4           20230103  09:00:04        SA305     2741.0  2741.600000   
...              ...       ...          ...        ...          ...   
9144627     20250815  14:59:55        SA601     1396.0  1398.351333   
9144628     20250815  14:59:56        SA601     1395.0  1398.351333   
9144629     20250815  14:59:57        SA601     1395.0  1398.351333   
9144630     20250815  14:59:58        SA601     1396.0  1398.351667   
9144631     20250815  14:59:59        SA601     1395.0  1398.351667   

          upper_band   lower_band     shortEMA      longEMA   ATR  BidPrice1  \
0                NaN   

C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos'] = 0  # 0表示空仓，1表示持多仓，-1表示持空仓
C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cost'] = np.nan  # 开仓成本价
C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

Processing parameter set 1/3: 0.004, 2.5, 1.2, 10
正在创建5分钟K线...
正在准备数据以便计算...
正在计算S_Trend...
正在计算shortTrendScore...
计算完成。


...

C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos'] = 0  # 0表示空仓，1表示持多仓，-1表示持空仓
C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cost'] = np.nan  # 开仓成本价
C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

Processing parameter set 1/3: 0.004, 2.5, 1.2, 10
正在创建5分钟K线...
正在准备数据以便计算...
正在计算S_Trend...
正在计算shortTrendScore...
计算完成。


C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos'] = 0  # 0表示空仓，1表示持多仓，-1表示持空仓
C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cost'] = np.nan  # 开仓成本价
C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

Processing parameter set 2/3: 0.006, 3, 1.5, 5
正在创建5分钟K线...
正在准备数据以便计算...
正在计算S_Trend...
正在计算shortTrendScore...
计算完成。


C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos'] = 0  # 0表示空仓，1表示持多仓，-1表示持空仓
C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cost'] = np.nan  # 开仓成本价
C:\Users\cyw\AppData\Local\Temp\ipykernel_45352\291542341.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c